In [1]:
import os
import gc
import ast
import random
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import torchaudio
import IPython.display as ipd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models

import warnings
warnings.filterwarnings('ignore')

/home/mk8574/.conda/envs/mk8574_3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [70]:
class config:
    seed=3
    sample_rate= 16000
    n_fft=1024
    hop_length=512
    n_mels=64
    duration=5
    num_classes = 6
    train_batch_size = 32
    valid_batch_size = 64
    model_name = 'resnet50'
    epochs = 20
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    learning_rate = 1e-5

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(config.seed)

In [5]:
mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate=config.sample_rate, 
                                                      n_fft=config.n_fft, 
                                                      hop_length=config.hop_length, 
                                                      n_mels=config.n_mels)

In [47]:
df = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/data/train.csv")
df["label"].describe()

count    5001.000000
mean        2.454509
std         1.714637
min         0.000000
25%         1.000000
50%         2.000000
75%         4.000000
max         5.000000
Name: label, dtype: float64

In [64]:
class AudioSentDataset(Dataset):
    def __init__(self, df, transformation, target_sample_rate, duration ,mode='train'):
        self.audio_paths = df['path'].values
        self.labels = df['label'].values
        self.transformation = transformation
        self.target_sample_rate = target_sample_rate
        self.num_samples = target_sample_rate*duration
        self.mode = mode
        
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, index):
        
        audio_path = f'/scratch/network/mk8574/audio_sentiment_challenge/data/{self.audio_paths[index]}'
        
        signal, sr = torchaudio.load(audio_path) # loaded the audio
        
        # Now we first checked if the sample rate is same as TARGET_SAMPLE_RATE and if it not equal we perform resampling
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        
        # Next we check the number of channels of the signal
        #signal -> (num_channels, num_samples) - Eg.-(2, 14000) -> (1, 14000)
        if signal.shape[0]>1:
            signal = torch.mean(signal, axis=0, keepdim=True)
        
        # Lastly we check the number of samples of the signal
        #signal -> (num_channels, num_samples) - Eg.-(1, 14000) -> (1, self.num_samples)
        # If it is more than the required number of samples, we truncate the signal
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        
        # If it is less than the required number of samples, we pad the signal
        if signal.shape[1]<self.num_samples:
            num_missing_samples = self.num_samples - signal.shape[1]
            last_dim_padding = (0, num_missing_samples)
            signal = F.pad(signal, last_dim_padding)
        
        # Finally all the process has been done and now we will extract mel spectrogram from the signal
        mel = self.transformation(signal)
        
        # For pretrained models, we need 3 channel image, so for that we concatenate the extracted mel
        image = torch.cat([mel, mel, mel])
        
        # Normalized the image(그냥 나누기)
        max_val = torch.abs(image).max()
        image = abs(image) / max_val
        
        label = torch.tensor(self.labels[index])
        if self.mode in ["train","valid"]:
            return image, label
        elif self.mode in ["test"]:
            return image
        else:
            assert False, f"Invalid mode : {self.mode}"

In [65]:
from sklearn.model_selection import train_test_split

def get_data():
    train_df,valid_df = train_test_split(df, test_size=0.2, random_state=config.seed, shuffle=True)
    
    train_dataset = AudioSentDataset(train_df, mel_spectrogram, config.sample_rate, config.duration,mode = "train")
    valid_dataset = AudioSentDataset(valid_df, mel_spectrogram, config.sample_rate, config.duration,mode = "valid")
    
    train_loader = DataLoader(train_dataset, batch_size=config.train_batch_size, shuffle=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=config.valid_batch_size, shuffle=False,drop_last=True)
    
    return train_loader, valid_loader

In [66]:
class BirdCLEFResnet(nn.Module):
    def __init__(self):
        super(BirdCLEFResnet, self).__init__()
        self.base_model = models.__getattribute__(config.model_name)(pretrained=True)#True
        for param in self.base_model.parameters():
            param.requires_grad = False
            
        in_features = self.base_model.fc.in_features
        
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features, config.num_classes))
        
    def forward(self, x):
        x = self.base_model(x)
        x.clamp(min=1e-8)
        return x


In [ ]:
def loss_fn(outputs, labels):
    return nn.CrossEntropyLoss(label_smoothing=0.001)(outputs, labels)

def train(model, data_loader, optimizer, scheduler, device, epoch):
    model.train()
    
    running_loss = 0
    loop = tqdm(data_loader, position=0)
    for i, (mels, labels) in enumerate(loop):
        mels = mels.to(device)
        labels = labels.to(device)
        
        outputs = model(mels)
       
        _, preds = torch.max(outputs, 1)
        
        loss = loss_fn(outputs, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()
        optimizer.zero_grad()
        
        if scheduler is not None:
            scheduler.step()
            
        
        running_loss += loss.item()
        #rint(loss.item(),running_loss,end="-")
        loop.set_description(f"Epoch [{epoch+1}/{config.epochs}]")
        loop.set_postfix(loss=loss.item())
   
    return running_loss/len(data_loader)

In [18]:
def valid(model, data_loader, device, epoch):
    model.eval()
    
    running_loss = 0
    pred = []
    label = []
    
    loop = tqdm(data_loader, position=0)
    for mels, labels in loop:
        mels = mels.to(device)
        labels = labels.to(device)
        
        outputs = model(mels)
        _, preds = torch.max(outputs, 1)
        
        loss = loss_fn(outputs, labels)
            
        running_loss += loss.item()
        
        pred.extend(preds.view(-1).cpu().detach().numpy())
        label.extend(labels.view(-1).cpu().detach().numpy())
        
        loop.set_description(f"Epoch [{epoch+1}/{config.epochs}]")
        loop.set_postfix(loss=loss.item())
        
    #valid_f1 = f1_score(label, pred, average='macro')
    accuracy = sum(1 for x,y in zip(label,pred) if x == y) / len(pred)
    return running_loss/len(data_loader), accuracy

In [68]:
import datetime
def run():
    now = datetime.datetime.now()
    nowDatetime = now.strftime('%Y%m%d%H%M%S')
    train_loader, valid_loader = get_data()
    
    #model = BirdClefModel().to(config.device) # check version 3 for this
    model = BirdCLEFResnet().to(config.device)
    
    optimizer = Adam(model.parameters(), lr=config.learning_rate)
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=1e-5, T_max=10)
    
    best_valid_f1 = 0
    for epoch in range(config.epochs):
        train_loss = train(model, train_loader, optimizer, scheduler, config.device, epoch)
        valid_loss, valid_f1 = valid(model, valid_loader, config.device, epoch)
        print(train_loss,valid_loss)
        if valid_f1 > best_valid_f1:
            print(f"Validation Accuracy Improved - {best_valid_f1} ---> {valid_f1}")
            torch.save(model.state_dict(), f'./model_{nowDatetime}.bin')
            print(f"Saved model checkpoint at ./model_{nowDatetime+str(epoch)}.bin")
            best_valid_f1 = valid_f1
        else:
            print(f"Validation Score : BEST: {best_valid_f1} NOW: {valid_f1}")
            
    return best_valid_f1

In [73]:


best_valid_f1 = run()
print(f'Best ACCURACY: {best_valid_f1:.5f}')

gc.collect()
torch.cuda.empty_cache()    
 # To run for all the folds, just remove this break

Epoch [1/20]:   6%|▋         | 8/125 [00:03<00:57,  2.03it/s, loss=1.8] 


KeyboardInterrupt: 